In [7]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [87]:
features_30s = pd.read_csv("../Data/features_30_sec.csv")

features_30s.iloc[3, 1:-1]

length                             661794
chroma_stft_mean                 0.404785
chroma_stft_var                  0.093999
rms_mean                         0.141093
rms_var                          0.006346
spectral_centroid_mean        1070.106615
spectral_centroid_var       184355.942417
spectral_bandwidth_mean       1596.412872
spectral_bandwidth_var      166441.494769
rolloff_mean                  2184.745799
rolloff_var                1493194.360296
zero_crossing_rate_mean          0.033309
zero_crossing_rate_var           0.000423
harmony_mean                          0.0
harmony_var                      0.019054
perceptr_mean                   -0.000014
perceptr_var                     0.002712
tempo                           63.024009
mfcc1_mean                    -199.544205
mfcc1_var                      5507.51709
mfcc2_mean                     150.090897
mfcc2_var                      456.505402
mfcc3_mean                       5.662678
mfcc3_var                      257

In [82]:
import torch.nn.functional as F

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.label_idx = {label: idx for idx, label in enumerate(sorted(set(self.data["label"])))}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        input = torch.tensor(self.data.iloc[idx, 2:-1], dtype=torch.float32)
        label = F.one_hot(torch.tensor(self.label_idx[self.data.iloc[idx, -1]]), num_classes=len(self.label_idx))
        
        if self.transform:
            input = self.transform(input)

        return input, label

In [83]:
dataset = CustomDataset("../Data/features_30_sec.csv")

In [85]:
inp, label = dataset[0]
len(inp)

/tmp/ipykernel_2106/1103232190.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  input = torch.tensor(self.data.iloc[idx, 2:-1], dtype=torch.float32)


57

In [93]:
from torch.utils.data.sampler import SubsetRandomSampler

torch.manual_seed(123)

train_sampler = SubsetRandomSampler(range(1000))
train_loader = torch.utils.data.DataLoader(dataset, batch_size=1, num_workers=1, sampler=train_sampler)

In [95]:
for i, data in enumerate(train_loader, 0):
    # Get the inputs
    inputs, labels = data
    print(inputs[0], labels)
    break

/tmp/ipykernel_2106/1103232190.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  input = torch.tensor(self.data.iloc[idx, 2:-1], dtype=torch.float32)


tensor([ 4.0253e-01,  8.8676e-02,  7.9205e-02,  2.4827e-04,  2.6692e+03,
         4.5984e+05,  2.4403e+03,  8.5044e+04,  5.4459e+03,  1.3686e+06,
         1.3342e-01,  4.5573e-03,  6.3759e-04,  4.0046e-03,  1.2969e-03,
         9.5731e-04,  1.5200e+02, -1.2333e+02,  2.6374e+03,  7.4883e+01,
         6.3775e+02, -1.7065e+01,  2.7358e+02,  3.1612e+01,  2.0048e+02,
         2.5440e+00,  1.0802e+02,  1.7331e+01,  8.9299e+01, -3.8127e+00,
         1.0200e+02,  9.7295e+00,  6.1421e+01, -7.2345e+00,  6.6062e+01,
         1.1267e+01,  4.7011e+01,  3.7987e-01,  4.4066e+01,  3.4734e+00,
         4.9047e+01,  1.6684e+00,  7.0591e+01, -3.1595e+00,  4.9460e+01,
        -1.0819e+00,  4.4166e+01,  3.1769e-01,  4.3706e+01, -5.9882e+00,
         5.0242e+01, -3.2017e-01,  7.2723e+01, -2.3397e+00,  6.5235e+01,
        -4.3212e+00,  5.0373e+01]) tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])
